In [2]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
import faiss
import torch
import numpy as np

In [3]:
%%capture --no-stdout
context_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokeniz

In [4]:
with open('../the-verdict.txt', 'r') as file:
    corpus = file.readlines()

In [5]:
encoded_corpus = []
for text in corpus:
    inputs = tokenizer(text, return_tensors='pt', truncation=True)
    outputs = context_encoder(**inputs)
    embedding = outputs.pooler_output.squeeze(0).detach().numpy()
    encoded_corpus.append(embedding)

encoded_corpus = np.array(encoded_corpus)
d = encoded_corpus.shape[1]
index = faiss.IndexFlatL2(d)
index.add(encoded_corpus)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [6]:
encoded_corpus.shape

(165, 768)

In [7]:
query = "What is the nature of existence?"
query_inputs = tokenizer(query, return_tensors='pt')
query_embedding = context_encoder(**query_inputs).pooler_output.squeeze(0).detach().numpy()

In [8]:
D, I = index.search(np.array([query_embedding]), k=1) # try other k's

Error: Canceled future for execute_request message before replies were done

In [ ]:
print("Indices of Retrieved Documents:", I)
print("Distances of Retrieved Documents:", D)

In [ ]:
retrieved_docs = [corpus[i] for i in I[0]]
retrieved_docs